# Projekt Zdjęcie Panoramiczne
przedmiot: Algebra Liniowa

pracownia specjalistyczna 9

członkowie zespołu:
- Jan Hapunik - 120316
- Izabela Rybińska - uzupełnij:3

## 1. wprowadzenie teoretyczne oraz opis wykorzystanego aparatu matematycznego

Zadanie polega na stworzeniu programu będącego w stanie połaczyć przynajmniej dwa zdjęcia wykonane z tego samego miejsca, opisujące różne przestrzenie (ale z istniejącymi punktami wspólnymi) w jedno zdjęcie panoramiczne przy użyciu metod z algebry liniowej. 

### **Aby to osiągnąć potrzebujemy:**

- dwa zdjęcia, z zaznaczonymi punktami wspólnymi, wyznaczonymi ręcznie bądź przez komputer, na początku jednak zrobimy to ręcznie;
- kod umożliwiający przekształcenie zdjęć na osi trzech współrzędnych tak, by zaznaczone punkty się na siebie nakładały. Trzecia współrzędna jest używana jako waga, pozwalająca skalować obraz, więc tak naprawdę poruszamy się po osi dwóch głównych współrzędnych i jednej pomocnej;
- przycięcie końcowego zdjęcia do odpowiednich rozmiarów;

### **Część teoretyczna:**

**Narzędzia matematyczne, z których będziemy korzystać:**

- **współrzędne jednorodne**

Punkt na obrazie wejściowo opisujemy jako:
$$ p = (x, y, w)^T $$
Niewiadome te reprezentują punkt kartezjański:
$$ (\frac{x}{w}, \frac{y}{w}) $$
Wartość w (na wejściu równa 1) pełni rolę współczynnika skali, dlatego różne trójki jednorodne mogą opisywać ten sam punkt w przestrzeni euklidesowej.
Zastosowanie współrzędnych jednorodnych pozwala zapisać przekształcenia perspektywiczne (homografie) w postaci liniowej jako mnożenie przez macierz 
3 x 3, co nie byłoby możliwe w klasycznym układzie kartezjańskim.

- **homografia**

WIP


## 2. wprowadzenie techniczne zawierające opis bibliotek oraz sposób rozwiązania zagadnienia:

In [4]:
# import potrzebnych bibliotek

from PIL import Image
# PIL (Pillow) pozwala na otworzenie i przygotowanie zdjęć wejściowych do dalszego przetwarzania

import matplotlib.pyplot as plt
# Matplotlib pozwala na wyświetlanie obrazów, prezentację wyników po przekształceniach
import numpy as np
# Numpy potrzebne jest do transformacji i obliczeń macierzy

proces: 
1. w
2. i
3. p


## 3. rozwiązania poszczególnych zadań projektowych;

Punkty kontrolne: 

1. Samodzielnie wykonać co najmniej dwa różne zdjęcia; wykonane z tego samego miejsca,
opisujące różne przestrzenie, których przecięcie jest niepuste.
2. Z powyższych zdjęć skonstruować jedno zdjęcie panoramiczne korzystając z metod algebry
liniowej.
(a) Sklasyfikować i sformułować formalnie wszystkie wykonywane przekształcenia oraz
metody algebry liniowej.
(b) Opisać matematycznie wszystkie rzuty wraz z formalnym przekształceniem.
(c) Rozrysować oba zdjęcia w odpowiednim dla nich układzie współrzędnych.
3. Czy istnieją metody automatycznej klasyfikacji punktów zbieżnych na obu fotografiach?
4. Jaki maksymalnie kąt możemy uchwycić korzystając z odpowiedniego rzutu na płaszczyznę?
Dlaczego?

## 4. podsumowanie wyników oraz wnioski;

## 5. notatkę zawierającą informację, które z zadań było najtrudniejsze.